In [1]:
import glob
g = glob.glob('data/processed_games/[0-9]*.csv')

In [ ]:
bigram_cts = None
for n in g:
    print (n)
    df = pd.read_csv(n)
    df_sorted = df.sort_values('game_clock')
    df_sorted['court_location'] = df_sorted['court_location'].apply(lambda x: (x,) if x is not None else None)

    grouped = df_sorted.groupby(['game_clock'])
    df_sorted = grouped.apply(get_rel_tobball)
#     print(df_sorted['court_location'])
    df_sorted['player_ball'] = df_sorted['ball_loc']+df_sorted['court_location']
    df_sorted = df_sorted.groupby(['player_id', 'event_id']).apply(make_bigrams_player_ball)

    if bigram_cts is None:
        df_sorted = df_sorted[['player_id', 'GAME_ID',
                                'bigram_player_ball']].dropna()
        df_sorted['bigram_player_ball_0'], df_sorted['bigram_player_ball_1'] = zip(*df_sorted['bigram_player_ball'].tolist())
        # need to split into indices since pandas can't handle nested tuples
        bigram_cts = df_sorted.groupby(['player_id', 'GAME_ID', 'bigram_player_ball_0', 'bigram_player_ball_1']).count()        
    else:
        df_sorted = df_sorted[['player_id', 'GAME_ID',
                                'bigram_player_ball']].dropna()
        df_sorted['bigram_player_ball_0'], df_sorted['bigram_player_ball_1'] = zip(*df_sorted['bigram_player_ball'].tolist())
        _cts = df_sorted.groupby(['player_id', 'GAME_ID', 'bigram_player_ball_0', 'bigram_player_ball_1']).count()
        bigram_cts = bigram_cts.add(_cts, fill_value=0.0)
        print(len(bigram_cts.index))
        
    del df

    